In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import pickle
import tqdm

In [2]:
num_special_tokens = 4
max_seq_len = 15

In [3]:
with open('./user_history_data.pickle', 'rb') as f:
    user_history_data = pickle.load(f)

In [4]:
user_history_data[4]

[{'id': 426345,
  'time': 1610640584518.0,
  'lane': 8,
  'item': 81,
  'bans': [17, 85, 25, 42, 84, 28, 60, 78, 63, 48],
  'data_type': 'train',
  'win': 4},
 {'id': 426335,
  'time': 1610642709578.0,
  'lane': 6,
  'item': 56,
  'bans': [63, 7, 25, 8, 32, 59, 120, 3, 3, 95],
  'data_type': 'train',
  'win': 4},
 {'id': 356958,
  'time': 1611474267605.0,
  'lane': 5,
  'item': 32,
  'bans': [97, 21, 11, 35, 37, 3, 21, 85, 42, 12],
  'data_type': 'train',
  'win': 4},
 {'id': 356947,
  'time': 1611475778191.0,
  'lane': 5,
  'item': 118,
  'bans': [42, 46, 14, 86, 25, 69, 21, 45, 17, 35],
  'data_type': 'train',
  'win': 5},
 {'id': 1037244,
  'time': 1611478680210,
  'lane': 6,
  'item': 16,
  'bans': [19, 3, 28, 21, 56, 14, 29, 25, 120, 47],
  'data_type': 'train',
  'win': 5},
 {'id': 1037237,
  'time': 1611479847907,
  'lane': 8,
  'item': 16,
  'bans': [10, 19, 86, 120, 21, 3, 56, 42, 3, 28],
  'data_type': 'train',
  'win': 4},
 {'id': 244267,
  'time': 1611483282008.0,
  'lane':

## User

### Q1: does win-rate of the previous match affects the current game?

In [5]:
win_df = pd.DataFrame(columns = ['prev_win','next_win'])

In [6]:
prev_wins = []
next_wins = []
for user_idx in tqdm.tqdm(range(len(user_history_data))):
    user_history = user_history_data[user_idx]
    
    prev_win = 0
    next_win = 0
    for history_idx in range(len(user_history)):
        history = user_history[history_idx]
        if history_idx == 0:
            prev_win = history['win'] - num_special_tokens
            continue
        next_win = history['win'] - num_special_tokens
        prev_wins.append(prev_win)
        next_wins.append(next_win)
        prev_win = next_win
        
win_df['prev_win'] = prev_wins
win_df['next_win'] = next_wins

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 162993/162993 [00:03<00:00, 53154.47it/s]


In [7]:
cf_matrix = pd.crosstab(win_df['next_win'], win_df['prev_win'], 
                        rownames=['next'], colnames=['prev'])
cf_matrix[0] = cf_matrix[0] / sum(cf_matrix[0])
cf_matrix[1] = cf_matrix[1] / sum(cf_matrix[1])
cf_matrix

prev,0,1
next,,
0,0.497028,0.484739
1,0.502972,0.515261


### Q2: does win-rate of the previous matches (win-rate of recent 15 games) affects the current game?

In [8]:
win_df = pd.DataFrame(columns = ['prev_win','next_win'])

In [9]:
prev_wins = []
next_wins = []
for user_idx in tqdm.tqdm(range(len(user_history_data))):
    user_history = user_history_data[user_idx]
    
    win_list = []
    for history_idx in range(len(user_history)):
        history = user_history[history_idx]
        win_list.append(history['win'] - num_special_tokens)
    
    for slide_idx in range(1, len(win_list)):
        start_idx = max(0, slide_idx - max_seq_len)
        prev_win = np.mean(win_list[start_idx:slide_idx]) >= 0.5
        next_win = win_list[slide_idx]
        
        prev_wins.append(prev_win)
        next_wins.append(next_win)

win_df['prev_win'] = prev_wins
win_df['next_win'] = next_wins

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 162993/162993 [01:21<00:00, 2008.02it/s]


In [10]:
cf_matrix = pd.crosstab(win_df['next_win'], win_df['prev_win'], 
                        rownames=['next'], colnames=['prev'])
cf_matrix[0] = cf_matrix[0] / sum(cf_matrix[0])
cf_matrix[1] = cf_matrix[1] / sum(cf_matrix[1])
cf_matrix

prev,False,True
next,,
0,0.498079,0.48498
1,0.501921,0.51502


### Q3: if the users pick their most-picked champion, is it likely to win?

In [11]:
win_df = pd.DataFrame(columns = ['most_frequent_picks','next_win'])

In [12]:
most_frequent_picks = {}
next_wins = {}
max_seq_lens = [1, 5, 10, 20, 50, 100]

for seq_len in max_seq_lens:
    most_frequent_picks[seq_len] = []
    next_wins[seq_len] = []

for user_idx in tqdm.tqdm(range(len(user_history_data))):
    user_history = user_history_data[user_idx]
    
    win_list = []
    item_list = []
    for history_idx in range(len(user_history)):
        history = user_history[history_idx]
        win_list.append(history['win'] - num_special_tokens)
        item_list.append(history['item'])
    
    for slide_idx in range(1, len(win_list)):
        for seq_len in max_seq_lens:
            start_idx = max(0, slide_idx - seq_len)
            prev_items = item_list[start_idx:slide_idx]
            item_counts = np.bincount(prev_items)
            most_frequent_item = np.argmax(item_counts)
            cur_item = item_list[slide_idx]
            if most_frequent_item == cur_item:
                most_frequent_picks[seq_len].append(1)
            else:
                most_frequent_picks[seq_len].append(0)
            next_win = win_list[slide_idx]
            next_wins[seq_len].append(next_win)

#win_df['most_frequent_picks'] = most_frequent_picks
#win_df['next_win'] = next_wins

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 162993/162993 [04:13<00:00, 643.79it/s]


In [13]:
for seq_len in max_seq_lens:
    win_df = pd.DataFrame(columns = ['most_frequent_picks','next_win'])
    win_df['most_frequent_picks'] = most_frequent_picks[seq_len]
    win_df['next_win'] = next_wins[seq_len]
    cf_matrix = pd.crosstab(win_df['next_win'], win_df['most_frequent_picks'], 
                            rownames=['next'], colnames=['prev'])
    cf_matrix[0] = cf_matrix[0] / sum(cf_matrix[0])
    cf_matrix[1] = cf_matrix[1] / sum(cf_matrix[1])

    print('seq_len:', seq_len)
    print('most frequency ratio:', np.mean(most_frequent_picks[seq_len]))
    print(cf_matrix, end='\n\n')

seq_len: 1
most frequency ratio: 0.3368542966113968
prev         0        1
next                   
0     0.499196  0.47412
1     0.500804  0.52588

seq_len: 5
most frequency ratio: 0.36777840254179833
prev         0         1
next                    
0     0.501856  0.471655
1     0.498144  0.528345

seq_len: 10
most frequency ratio: 0.37283134810795554
prev       0         1
next                  
0     0.5028  0.470477
1     0.4972  0.529523

seq_len: 20
most frequency ratio: 0.3692595632710618
prev         0         1
next                    
0     0.503336  0.469248
1     0.496664  0.530752

seq_len: 50
most frequency ratio: 0.36247600526207385
prev         0         1
next                    
0     0.503512  0.468301
1     0.496488  0.531699

seq_len: 100
most frequency ratio: 0.3603097418075933
prev         0         1
next                    
0     0.503532  0.468054
1     0.496468  0.531946



In [14]:
print('Optimal train accuracy:', 0.36 * 0.532 + 0.64 * 0.503)

Optimal train accuracy: 0.51344


## Champion

In [15]:
from collections import defaultdict

### Pick-rate and win-rate of each champion

In [16]:
items_win = defaultdict(list)
items_cnt = defaultdict(list)

total_cnt = 0
for user_idx in tqdm.tqdm(range(len(user_history_data))):
    user_history = user_history_data[user_idx]
    
    win_list = []
    item_list = []
    for history_idx in range(len(user_history)):
        history = user_history[history_idx]
        
        item = history['item']
        items_win[item].append(history['win'] - num_special_tokens)
        items_cnt[item].append(1)
        total_cnt += 1

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 162993/162993 [00:03<00:00, 43671.50it/s]


In [17]:
Expected_acc = 0.0

for item, values in items_win.items():
    win_list = items_win[item]
    cnt_list = items_cnt[item]
    
    print('item:%3d win:%.2f cnt:%.2f' %(item, np.mean(win_list), np.sum(cnt_list) / total_cnt*100))
    Expected_acc += max(np.mean(win_list), 1- np.mean(win_list)) * (np.sum(cnt_list) / total_cnt)

print('Expected accuracy:', Expected_acc)

item: 81 win:0.53 cnt:1.24
item: 56 win:0.51 cnt:1.05
item: 32 win:0.48 cnt:1.50
item:118 win:0.49 cnt:0.32
item: 16 win:0.52 cnt:1.49
item: 53 win:0.52 cnt:0.90
item: 54 win:0.49 cnt:1.74
item:100 win:0.51 cnt:1.02
item: 48 win:0.54 cnt:0.61
item: 25 win:0.48 cnt:1.62
item: 71 win:0.52 cnt:0.31
item: 35 win:0.53 cnt:2.64
item: 17 win:0.54 cnt:1.36
item: 73 win:0.48 cnt:1.25
item:150 win:0.51 cnt:0.12
item:  4 win:0.51 cnt:0.80
item:  5 win:0.52 cnt:0.49
item: 95 win:0.51 cnt:1.31
item: 47 win:0.48 cnt:1.60
item: 84 win:0.50 cnt:0.52
item:  6 win:0.48 cnt:0.96
item: 13 win:0.50 cnt:1.25
item: 70 win:0.51 cnt:1.33
item: 59 win:0.50 cnt:1.73
item: 85 win:0.55 cnt:0.79
item: 43 win:0.50 cnt:1.23
item:125 win:0.51 cnt:0.22
item: 21 win:0.50 cnt:1.30
item:  9 win:0.52 cnt:1.96
item: 64 win:0.51 cnt:1.68
item:  7 win:0.54 cnt:4.98
item: 78 win:0.48 cnt:0.94
item: 38 win:0.50 cnt:0.63
item: 19 win:0.55 cnt:0.87
item:102 win:0.49 cnt:0.13
item: 98 win:0.48 cnt:0.10
item: 94 win:0.52 cnt:0.20
i

### Pick-rate and win-rate of combinations of champions

In [18]:
with open('./match_data.pickle', 'rb') as f:
    match_data = pickle.load(f)

In [19]:
def create_chmp_lane_ratio(dataset):
    def create_champ_lane_info(dataset):
        num_participants = 10
        train_champion_tmp = []
        train_lane_tmp = []

        for participant_id in range(1, num_participants + 1):
            train_champion_tmp.append(dataset['User' + str(participant_id) + '_champion'].to_numpy())
            train_lane_tmp.append(dataset['User' + str(participant_id) + '_lane'].to_numpy())

        # Get Each Champion's lane count
        train_champion = train_champion_tmp[0]
        train_lane = train_lane_tmp[0]

        for i in range(len(train_champion_tmp) - 1):
            train_champion = np.append(train_champion, train_champion_tmp[i + 1])
            train_lane = np.append(train_lane, train_lane_tmp[i + 1])
        return train_champion, train_lane

    train_champion, train_lane = create_champ_lane_info(dataset)

    def create_ratio_dict(train_champion, train_lane):
        train_champion_lane_dict = {}
        for i in range(train_champion.shape[0]):
            champ_idx = train_champion[i]
            lane_idx = train_lane[i]
            if champ_idx not in train_champion_lane_dict:
                train_champion_lane_dict[champ_idx] = {}
                for i in range(9):
                    train_champion_lane_dict[champ_idx][i] = 0
            train_champion_lane_dict[champ_idx][lane_idx] += 1
        for i in train_champion_lane_dict.keys():
            sum = 0
            for j in range(len(train_champion_lane_dict[i])):
                sum += train_champion_lane_dict[i][j]
            train_champion_lane_dict[i]['sum'] = sum
            for j in range(len(train_champion_lane_dict[i]) - 1):
                train_champion_lane_dict[i][j] = train_champion_lane_dict[i][j] / sum
        return train_champion_lane_dict

    train_champion_lane_dict = create_ratio_dict(train_champion, train_lane)
    del train_champion, train_lane
    return train_champion_lane_dict


def fill_missing_lane(dataset, train_champion_lane_dict):
    lane_df = dataset.filter(regex='lane')
    champ_df = dataset.filter(regex='champ')
    team1_lane_champ_df = pd.concat([lane_df.iloc[:, :5], champ_df.iloc[:, :5]], axis=1)
    team2_lane_champ_df = pd.concat([lane_df.iloc[:, 5:], champ_df.iloc[:, 5:]], axis=1)
    team1_lane_champ_np = team1_lane_champ_df.to_numpy()
    team2_lane_champ_np = team2_lane_champ_df.to_numpy()

    def get_missing_rows(team1_lane_champ_np, team2_lane_champ_np):

        # Get the problematic rows for each team individually
        team1_problematic_rows = []
        for i in range(team1_lane_champ_np.shape[0]):
            team1_lane_set = set((4, 5, 6, 7, 8))
            for j in team1_lane_champ_np[i, :5]:
                if j in team1_lane_set:
                    team1_lane_set.remove(j)
            if len(team1_lane_set) != 0:
                team1_problematic_rows.append(i)

        team2_problematic_rows = []
        for i in range(team2_lane_champ_np.shape[0]):
            team2_lane_set = set((4, 5, 6, 7, 8))
            for j in team2_lane_champ_np[i, :5]:
                if j in team2_lane_set:
                    team2_lane_set.remove(j)
            if len(team2_lane_set) != 0:
                team2_problematic_rows.append(i)
        return team1_problematic_rows, team2_problematic_rows

    team1_problematic_rows, team2_problematic_rows = get_missing_rows(team1_lane_champ_np, team2_lane_champ_np)

    # consider only the problematic rows for computational ease
    team1_prob_df = team1_lane_champ_df.iloc[team1_problematic_rows]
    team1_prob_np = team1_prob_df.to_numpy()
    team2_prob_df = team2_lane_champ_df.iloc[team2_problematic_rows]
    team2_prob_np = team2_prob_df.to_numpy()

    def fill_missing_rows(team_prob_np, team_prob_df, train_champion_lane_dict):
        for i in range(team_prob_np.shape[0]):
            unique_vals_tmp, unique_idx, unique_counts = np.unique(team_prob_np[i, :5], False, True, True)

            duplicate_vals = set(unique_vals_tmp[np.where(unique_counts > 1)])

            # minus set(3) since 3==Unk token, and may look unique!
            unique_vals = set(unique_vals_tmp[np.where(unique_counts == 1)])

            # Initialize problematic_idx with idx of Unk tokens, if there are any
            problematic_idx = []
            UNK = 3
            if UNK in unique_vals:
                problematic_idx.append(np.where(team_prob_np[i, :5] == UNK))
                if UNK in duplicate_vals:
                    duplicate_vals.remove(UNK)
            for k in range(len(duplicate_vals)):
                problematic_idx.append(np.where(unique_idx == np.where(unique_counts > 1)[0][k]))

            # Now need to know what lanes are missing!
            # We can't know this by only from the duplicate vals since That lane itself may not exist at all
            missing_lanes = set((4, 5, 6, 7, 8)) - unique_vals
            missing_idx = np.concatenate(problematic_idx, axis=1).squeeze()

            # Now we know which lanes and which users are problematic
            missing_chmp = team_prob_np[i, 5:][missing_idx]

            probs = []
            combinations = {}
            for idx, cmb in enumerate(pm(missing_lanes)):
                combinations[idx] = cmb
                prob = 1
                for champ_idx, lane_idx in enumerate(cmb):
                    prob = prob * (train_champion_lane_dict[missing_chmp[champ_idx]][lane_idx] + 1e-4)
                probs.append(prob)
            probs = np.array(probs)
            max_prob_idx = probs.argmax()
            team_prob_df.iloc[i, missing_idx] = combinations[max_prob_idx]

        return team_prob_df
    team1_filled_df = fill_missing_rows(team1_prob_np, team1_prob_df, train_champion_lane_dict)
    team2_filled_df = fill_missing_rows(team2_prob_np, team2_prob_df, train_champion_lane_dict)

    team1_lane_champ_df.iloc[team1_problematic_rows] = team1_filled_df
    team2_lane_champ_df.iloc[team2_problematic_rows] = team2_filled_df

    # Modify original dataset with most probable lane permutation for missing lane values
    dataset[team1_lane_champ_df.columns[:5]] = team1_lane_champ_df[team1_lane_champ_df.columns[:5]]
    dataset[team2_lane_champ_df.columns[:5]] = team2_lane_champ_df[team2_lane_champ_df.columns[:5]]

    return dataset

In [20]:
train_champion_lane_dict = create_chmp_lane_ratio(match_data['train'])
match_data['train'] = fill_missing_lane(match_data['train'], train_champion_lane_dict)

In [21]:
match_data['train']

,win,version,time,User1_id,User1_lane,User1_champion,User1_ban,User1_team,User2_id,User2_lane,...,User1_history,User2_history,User3_history,User4_history,User5_history,User6_history,User7_history,User8_history,User9_history,User10_history
0,Fail,4,1.61262e+12,4,8,4,17,4,5,4,...,0,1,2,3,4,5,6,7,8,9
1,Win,4,1.61262e+12,14,8,14,3,4,15,7,...,10,11,12,13,14,15,16,17,18,19
2,Fail,4,1.61261e+12,23,4,23,100,4,24,5,...,20,21,22,23,24,25,26,27,28,29
3,Fail,4,1.61261e+12,32,4,28,42,4,33,8,...,30,31,32,33,34,35,36,37,38,39
4,Fail,4,1.61259e+12,41,8,35,59,4,42,4,...,40,41,42,43,44,45,46,47,48,49
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1047947,Fail,8,1.60888e+12,3,5,32,33,4,3,6,...,0,0,0,0,0,0,0,6703260,0,0
1047948,Fail,8,1.60887e+12,3,4,9,32,4,3,7,...,0,0,0,0,0,6703261,0,0,0,0
1047949,Win,8,1.60887e+12,3,5,41,53,4,3,4,...,0,0,6703262,0,0,0,0,0,0,0
1047950,Win,8,1.60812e+12,119009,5,52,32,4,117611,6,...,6703263,6703264,6703265,0,6703266,0,0,6703267,6703268,6703269


In [22]:
bot_comb = defaultdict(list)

for match_idx in tqdm.tqdm(range(len(match_data['train']))):
    match = match_data['train'].iloc[match_idx]
    
    win = int(match['win'] == 'Win')
    for i in range(1, 6):
        lane = match['User' + str(i) + '_lane']
        if lane == 7:
            carry = match['User' + str(i) + '_champion']
        elif lane == 8:
            support = match['User' + str(i) + '_champion']
    
    bot_comb[(carry, support)].append(win)
    
    for i in range(6, 11):
        lane = match['User' + str(i) + '_lane']
        if lane == 7:
            carry = match['User' + str(i) + '_champion']
        elif lane == 8:
            support = match['User' + str(i) + '_champion']
    
    bot_comb[(carry, support)].append(1- win)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1047952/1047952 [02:42<00:00, 6442.73it/s]


In [23]:
optimal_train_acc = 0.0

for item, values in bot_comb.items():
    win_rate = np.mean(values)
    freq_rate = len(values) / (len(match_data['train']) * 2)
    print('item:' + str(item) + 'win:%.2f cnt:%.3f' %(win_rate, freq_rate))
    optimal_train_acc += max(win_rate, 1- win_rate) * (freq_rate)

print('Optimal train accuracy:', optimal_train_acc)

item:(7, 4)win:0.51 cnt:0.017
item:(10, 11)win:0.44 cnt:0.002
item:(7, 14)win:0.54 cnt:0.020
item:(20, 22)win:0.52 cnt:0.001
item:(5, 24)win:0.56 cnt:0.001
item:(7, 26)win:0.50 cnt:0.005
item:(31, 29)win:0.51 cnt:0.002
item:(34, 16)win:0.46 cnt:0.001
item:(34, 35)win:0.50 cnt:0.003
item:(7, 11)win:0.52 cnt:0.008
item:(42, 43)win:0.47 cnt:0.002
item:(47, 35)win:0.50 cnt:0.021
item:(7, 24)win:0.51 cnt:0.008
item:(7, 53)win:0.54 cnt:0.013
item:(54, 24)win:0.49 cnt:0.002
item:(58, 35)win:0.57 cnt:0.000
item:(62, 60)win:0.46 cnt:0.001
item:(42, 64)win:0.51 cnt:0.017
item:(7, 65)win:0.49 cnt:0.006
item:(31, 67)win:0.50 cnt:0.001
item:(10, 24)win:0.47 cnt:0.002
item:(70, 72)win:0.48 cnt:0.001
item:(42, 35)win:0.50 cnt:0.012
item:(7, 16)win:0.49 cnt:0.007
item:(75, 14)win:0.51 cnt:0.009
item:(79, 64)win:0.50 cnt:0.000
item:(62, 4)win:0.42 cnt:0.000
item:(70, 35)win:0.53 cnt:0.007
item:(20, 11)win:0.44 cnt:0.001
item:(7, 35)win:0.55 cnt:0.033
item:(31, 11)win:0.49 cnt:0.002
item:(62, 35)win:0.5

In [24]:
bot_comb = defaultdict(list)

for match_idx in tqdm.tqdm(range(len(match_data['train']))):
    match = match_data['train'].iloc[match_idx]
    
    win = int(match['win'] == 'Win')
    for i in range(1, 6):
        lane = match['User' + str(i) + '_lane']
        if lane == 7:
            carry = match['User' + str(i) + '_champion']
        elif lane == 8:
            support = match['User' + str(i) + '_champion']
    
    team1 = (carry, support)
    
    for i in range(6, 11):
        lane = match['User' + str(i) + '_lane']
        if lane == 7:
            carry = match['User' + str(i) + '_champion']
        elif lane == 8:
            support = match['User' + str(i) + '_champion']
    
    team2 = (carry, support)
    if (team1, team2) in bot_comb:
        bot_comb[(team1, team2)].append(win)
    elif (team2, team1) in bot_comb:
        bot_comb[(team2, team1)].append(1-win)
    else:
        bot_comb[(team1, team2)].append(win)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1047952/1047952 [02:47<00:00, 6274.57it/s]


In [25]:
optimal_train_acc = 0.0
freq_th = 250
total_freq = 0

for item, values in bot_comb.items():
    win_rate = np.mean(values)
    freq = len(values)
    if freq < freq_th:
        continue
    else:
        print('item:' + str(item) + 'win:%.2f cnt:%.3f' %(win_rate, freq))
        optimal_train_acc += max(win_rate, 1- win_rate) * (freq)
        total_freq += freq
        
print('Optimal train accuracy:', optimal_train_acc / total_freq)

item:((47, 35), (7, 24))win:0.53 cnt:726.000
item:((70, 35), (7, 53))win:0.49 cnt:439.000
item:((47, 35), (7, 91))win:0.46 cnt:626.000
item:((70, 81), (42, 64))win:0.48 cnt:292.000
item:((47, 64), (7, 14))win:0.47 cnt:430.000
item:((47, 35), (7, 60))win:0.53 cnt:635.000
item:((7, 60), (42, 64))win:0.55 cnt:364.000
item:((7, 81), (47, 35))win:0.48 cnt:1041.000
item:((7, 64), (20, 35))win:0.46 cnt:461.000
item:((7, 73), (42, 64))win:0.44 cnt:635.000
item:((75, 14), (7, 81))win:0.45 cnt:415.000
item:((7, 66), (75, 14))win:0.51 cnt:260.000
item:((75, 26), (7, 35))win:0.27 cnt:286.000
item:((7, 4), (42, 35))win:0.43 cnt:607.000
item:((7, 64), (10, 35))win:0.48 cnt:365.000
item:((7, 64), (42, 73))win:0.57 cnt:356.000
item:((75, 64), (7, 16))win:0.59 cnt:277.000
item:((75, 64), (42, 14))win:0.44 cnt:263.000
item:((7, 35), (31, 14))win:0.52 cnt:609.000
item:((42, 64), (7, 14))win:0.48 cnt:1198.000
item:((42, 64), (7, 35))win:0.49 cnt:1585.000
item:((42, 64), (7, 4))win:0.54 cnt:670.000
item:((